<a href="https://colab.research.google.com/github/linusms/Hands-on/blob/main/chapter_6_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. moons 데이터셋에 결정 트리를 훈련시키고 튜닝하기

In [22]:
# 데이터셋 생성, train,test 데이터셋 분류

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

X, y=make_moons(n_samples=10000, noise=0.4)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [29]:
print(X_train, y_train)


[[-0.65737184  0.75578113]
 [ 2.1946086  -0.34718854]
 [ 1.02584037 -0.98042004]
 ...
 [-0.45982048  0.76889171]
 [-0.13168496 -0.80984747]
 [ 1.00088039  0.81169244]] [0 1 1 ... 0 1 0]


In [2]:
# 결정트리 분류기 생성, 임의의 파라미터로 학습

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree_clf=DecisionTreeClassifier(max_depth=5, max_leaf_nodes=10)

scores=cross_val_score(tree_clf,X_train, y_train, cv=5).mean()
print(scores)

0.845375


In [3]:
# GridSearchCV를 사용해 최적 파라미터 찾기

from sklearn.model_selection import GridSearchCV

tree_params=[{'max_depth':list(range(2,20)),'max_leaf_nodes':list(range(2,100)), 
              'min_samples_split':[2,3,4]}]
gridsearch=GridSearchCV(DecisionTreeClassifier(), tree_params, cv=3, verbose=1)
gridsearch.fit(X_train,y_train)

print(gridsearch.best_params_)
print(gridsearch.best_score_)

Fitting 3 folds for each of 5292 candidates, totalling 15876 fits
{'max_depth': 6, 'max_leaf_nodes': 21, 'min_samples_split': 2}
0.8537501829525622


In [5]:
# 최적 파라미터로 테스트 세트 평가 - 대략 
from sklearn.metrics import accuracy_score

tree_clf=DecisionTreeClassifier(max_leaf_nodes=11)
tree_clf.fit(X_train, y_train)

y_pred=tree_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8615

In [4]:
from sklearn.metrics import accuracy_score

y_pred = gridsearch.predict(X_test)
accuracy_score(y_test, y_pred)

0.862

In [ ]:
# 2. 단계를 따라 랜덤 포레스트 만들어보기

In [9]:
# 1000개의 미니 훈련 세트가 100개의 샘플을 담도록 쪼개기
# 100개로 훈련한 모델이 9900개의 샘플의 답을 예측함(성능 낮음)

from sklearn.model_selection import ShuffleSplit

rs=ShuffleSplit(n_splits=1000, random_state=42, train_size=0.01)
mini_sets=[]

for mini_train_index, mini_test_index in rs.split(X_train):
  X_mini_train=X_train[mini_train_index]
  y_mini_train=y_train[mini_train_index]
  mini_sets.append((X_mini_train, y_mini_train))


In [16]:
# 앞서 구한 최적의 결정 트리로 각 미니 훈련 세트의 훈련 진행

from sklearn.base import clone

forest = [clone(gridsearch.best_estimator_) for _ in range(1000)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7912655

In [31]:
# 1000개의 예측값들 중 가장 많이 나온 것만 취합
# 다수결 예측 분류기 생성 완료

# Y_pred : 예측값들 모은 배열
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

print(Y_pred.shape)

from scipy.stats import mode

# y_pred_majority_votes는 각 열(axis=0)에서 최빈값인 값들을 모아놓은 배열
# 즉 2000개의 테스트 세트에 대해 예측한 각 값들 중 각 트리(1000개)에서 최빈값들만 모아놓음 
# n_votes는 각 행에서 해당 최빈값이 얼마나 나왔는지 카운트 
# (ex. 테스트셋[0]의 클래스는 1000개의 트리 중 849개에서 1로 예측)
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)
print(y_pred_majority_votes, n_votes)

(1000, 2000)
[[1 1 0 ... 0 1 1]] [[849 961 908 ... 920 954 871]]


In [14]:
# 정확도가 앞서보다 조금 높게 나옴
# 이게 랜덤 포레스트의 기본 원리

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.865